In [1]:
import logging
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime
from geopy.distance import geodesic

In [2]:
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(message)s', level=logging.INFO)

In [3]:
logging.info("Starting the Step 3 Calculations")

2024-10-27 00:18:55,115 - INFO - Starting the Step 3 Calculations


### Reading all the focal brands

In [ ]:
with open('top_brands.pickle', 'rb') as file:
    focal_brands = pickle.load(file)

focal_brands

### Reading the visitation data for all the brands

In [ ]:
brands_visits = pd.read_csv('data/revision_visits_revenue_2019.csv')
brands_visits['brand_standard'] = brands_visits['brand'].apply(lambda x: x.strip().lower()) # For comparison with catalog.tsv
brands_visits['date'] = brands_visits['date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').date())
brands_visits = brands_visits.rename(columns={'brand': 'brand_visitation'})
# Dropping unused columns
brands_visits = brands_visits.drop(columns=['spend_by_day', 'lat', 'lon', 'brand_standard'])
# Setting date as the index since we will group by date for calculation of metrics
brands_visits = brands_visits.sort_values('date').set_index('date')
brands_visits.head()

### Reading the Data for Local Reviews of all brands having social data

In [ ]:
with open('brand_visit_local_reviews.pickle', 'rb') as file:
    brand_visit_local_reviews = pickle.load(file)

brand_visit_local_reviews = brand_visit_local_reviews.drop(columns=['spend_by_day', 'lat', 'lon', 'brand_standard', 'ID', 'Name',
                                                                    'Genre', 'Type', 'Classification', 'Status', 'Name_Standard'])

brand_visit_local_reviews = brand_visit_local_reviews.sort_values('date').set_index('date')
brand_visit_local_reviews = brand_visit_local_reviews.fillna(0)
brand_visit_local_reviews.head()

In [5]:
brand_visit_local_reviews_list = brand_visit_local_reviews['brand_visitation'].unique().tolist()

### Reading the neib distance for all the neighbors

In [ ]:
with open('neib_distance_km_brand.pkl', 'rb') as file:
    neib_distance_km = pickle.load(file)

# Excluding all the brands having more than 1 mile distance since we don't need them anyways for second-degree neighbor calculations
neib_distance_km = neib_distance_km[neib_distance_km['Distance_Km']<=16.0934]
neib_distance_km.head()

### Reading Spatial Distance

In [ ]:
distance_results = pd.read_csv('data/distance_results.csv')
distance_results.head()

Checking if only the focal brands are present in the FROM_Placekey

In [ ]:
from_place_key_distance_brands = brands_visits[brands_visits['PLACEKEY'].isin(distance_results['From_PLACEKEY'].to_list())]['brand_visitation'].unique()
from_place_key_distance_brands

Checking the unique brands present in TO_Placekey

In [9]:
to_place_key_distance_brands = brands_visits[brands_visits['PLACEKEY'].isin(distance_results['To_PLACEKEY'].to_list())]['brand_visitation'].unique()

### Reading Travel Time

In [ ]:
with open('data/travel_time.pkl', 'rb') as file:
    travel_time_dict = pickle.load(file)
    
travel_time_keys = list(travel_time_dict.keys())
from_keys = [key[0] for key in travel_time_keys]
to_keys = [key[1] for key in travel_time_keys]
time_minutes = list(travel_time_dict.values())
time_minutes = [int(time_inst.split(' ')[0]) for time_inst in time_minutes]

travel_time = pd.DataFrame({'From_PLACEKEY': from_keys, 'To_PLACEKEY': to_keys, 'Time_mins': time_minutes})
travel_time.head()

Checking if only the focal brands are present in the FROM_Placekey

In [ ]:
from_place_key_time_brands = brands_visits[brands_visits['PLACEKEY'].isin(travel_time['From_PLACEKEY'].to_list())]['brand_visitation'].unique()
from_place_key_time_brands

Checking the unique brands present in TO_Placekey

In [12]:
to_place_key_time_brands = brands_visits[brands_visits['PLACEKEY'].isin(travel_time['To_PLACEKEY'].to_list())]['brand_visitation'].unique()

### Comparing the brands in distance values and time values

In [ ]:
from_place_key_distance_brands.sort() == from_place_key_time_brands.sort()

In [ ]:
to_place_key_distance_brands.sort() == to_place_key_time_brands.sort()

It seems like both distance and time data have same brands in both the placekey and tokey values.

In [ ]:
print(len(to_place_key_distance_brands))
print(len(to_place_key_time_brands))

Since the total brands are only 418 which is close to what the brands we have containing local reviews (427), let's see if these are the same brands which are present in thos brands

In [16]:
local_reviews_brands = brand_visit_local_reviews['brand_visitation'].unique()

In [ ]:
print('Total Constant brands between to_placekey and brands having local reviews:', len(set(local_reviews_brands).intersection(set(to_place_key_distance_brands))))

Since almost half of the brands don't have local reviews, so we are only going to consider the Neighboring brands from distance df for all the focal brands

Let's see if the to_placekey contains any focal brand or not

In [ ]:
set(from_place_key_distance_brands).intersection(set(to_place_key_distance_brands))

So, now its clear that to_placekey also contains the focal brands. Now, we can go ahead and simply consider all the to_brands as the neighboring brands and from_key as the focal brands

### Performing First Degree Neighbor Calculations

Let's see all the focal brands we have

In [ ]:
focal_brands_list = focal_brands['BRANDS'].tolist()
focal_brands_list

Select a focal brand and then extract all the PlaceKeys for this focal brand

In [ ]:
foc_brand = focal_brands_list[0]
foc_brand

Getting all the store keys for this specific focal brand

In [ ]:
store_keys_foc_brand = brand_visit_local_reviews[brand_visit_local_reviews['brand_visitation'] == foc_brand]['PLACEKEY'].unique().tolist()
store_keys_foc_brand

Getting all the neighboring keys for each store of focal brand

In [ ]:
all_neib_placekey = distance_results[distance_results['From_PLACEKEY'].isin(store_keys_foc_brand)]['To_PLACEKEY'].unique().tolist()
all_neib_placekey

Getting Unique Neighbors for all stores of the focal brand

In [23]:
unique_neib_brands_foc = brands_visits[brands_visits['PLACEKEY'].isin(all_neib_placekey)]['brand_visitation'].unique().tolist()

In [ ]:
len(unique_neib_brands_foc)

1. For each Unique Neighboring Brand, get all the placekeys
2. Filter them as per Neib PlaceKeys for all the focal stores
3. For each focal store, calculate first neib metrics (with local reviews if it is present in local_reviews, else only calculate visits from brand_visits data)

In [ ]:
unique_neib = unique_neib_brands_foc[0]
unique_neib

In [ ]:
unique_neib_placekeys = brands_visits[brands_visits['brand_visitation'] == unique_neib]['PLACEKEY'].unique().tolist()
unique_neib_placekeys

In [ ]:
focal_stores_first_degree_neib = distance_results[(distance_results['From_PLACEKEY'].isin(store_keys_foc_brand)) &
                                                (distance_results['To_PLACEKEY'].isin(unique_neib_placekeys)) &
                                                (distance_results['Distance_km']<=16.0934)]
focal_stores_first_degree_neib

In [ ]:
store_keys_foc_brand = focal_stores_first_degree_neib['From_PLACEKEY'].unique().tolist()
foc_store = store_keys_foc_brand[0]
foc_store

Calculating all the brands for the stores which can be categorized as the first degreen neighbors. All the stores of such brands need to be excluded from second degree neighbor calculation

In [29]:
foc_store_all_first_degree_neibs = distance_results[(distance_results['From_PLACEKEY'] == foc_store) & (distance_results['Distance_km'] <= 16.0934)]['To_PLACEKEY'].to_list()
foc_store_all_first_degree_neibs_brands = brands_visits[brands_visits['PLACEKEY'].isin(foc_store_all_first_degree_neibs)]['brand_visitation'].unique().tolist()

In [ ]:
foc_store_first_degree_neibs = focal_stores_first_degree_neib[focal_stores_first_degree_neib['From_PLACEKEY'] == foc_store]['To_PLACEKEY'].to_list()
foc_store_first_degree_neibs

In [ ]:
foc_store_distance = distance_results[(distance_results['From_PLACEKEY'] == foc_store) & (distance_results['To_PLACEKEY'].isin(foc_store_first_degree_neibs))]
foc_store_distance

In [ ]:
spatial_distance_avg = np.average(foc_store_distance['Distance_km'].to_list())
spatial_distance_avg

In [ ]:
foc_store_time = travel_time[(travel_time['From_PLACEKEY'] == foc_store) & (travel_time['To_PLACEKEY'].isin(foc_store_first_degree_neibs))]
foc_store_time

In [ ]:
travel_distance_avg = np.average(foc_store_time['Time_mins'].to_list())
travel_distance_avg

If the unique neighbour in question is has the local reviews, then we need to calculate local neib metrics else only simple visits will be needed

In [35]:
def calculate_first_neib_mean_reviews_visits(group_df):
    
    inv_visits = 0
    inv_visits_exp = 0
    
    num_reviews_fb_neibmean = 0
    num_reviews_ig_neibmean = 0
    num_reviews_tw_neibmean = 0

    num_reviews_fb_neibmean_exp = 0
    num_reviews_ig_neibmean_exp = 0
    num_reviews_tw_neibmean_exp = 0
    
    
    for neib_store in foc_store_first_degree_neibs:
        neib_store_time = foc_store_time[(foc_store_time['From_PLACEKEY'] == foc_store) & (foc_store_time['To_PLACEKEY'] == neib_store)]['Time_mins'].values[0]
        
        neib_store_reviews = group_df[group_df['PLACEKEY'] == neib_store][['localized_fb_reviews_60_days',
                                                                            'localized_ig_reviews_60_days',
                                                                            'localized_tw_reviews_60_days',
                                                                            'visits_by_day']]
        
        fb_reviews = neib_store_reviews['localized_fb_reviews_60_days']
        if (len(fb_reviews) != 0):
            num_reviews_fb_neibmean += (1/neib_store_time) * fb_reviews.values[0]
            num_reviews_fb_neibmean_exp += (1/np.exp(neib_store_time)) * fb_reviews.values[0]
        
        ig_reviews = neib_store_reviews['localized_ig_reviews_60_days']
        if (len(ig_reviews) != 0):
            num_reviews_ig_neibmean += (1/neib_store_time) * ig_reviews.values[0]
            num_reviews_ig_neibmean_exp += (1/np.exp(neib_store_time)) * ig_reviews.values[0]
        
        tw_reviews = neib_store_reviews['localized_tw_reviews_60_days']
        if (len(tw_reviews) != 0):
            num_reviews_tw_neibmean += (1/neib_store_time) * tw_reviews.values[0]
            num_reviews_tw_neibmean_exp += (1/np.exp(neib_store_time)) * tw_reviews.values[0]
        
        visits = neib_store_reviews['visits_by_day']
        if (len(visits) != 0):
            inv_visits += (1/neib_store_time) * visits.values[0]
            inv_visits_exp += (1/np.exp(neib_store_time)) * visits.values[0]
    
    return pd.Series([foc_store, inv_visits, num_reviews_fb_neibmean, num_reviews_ig_neibmean, num_reviews_tw_neibmean,
                      inv_visits_exp, num_reviews_fb_neibmean_exp, num_reviews_ig_neibmean_exp, num_reviews_tw_neibmean_exp])

In [36]:
def calculate_first_neib_visits(group_df):
    
    inv_visits = 0
    inv_visits_exp = 0
    
    num_reviews_fb_neibmean = 0
    num_reviews_ig_neibmean = 0
    num_reviews_tw_neibmean = 0

    num_reviews_fb_neibmean_exp = 0
    num_reviews_ig_neibmean_exp = 0
    num_reviews_tw_neibmean_exp = 0
    
    
    for neib_store in foc_store_first_degree_neibs:
        neib_store_time = foc_store_time[(foc_store_time['From_PLACEKEY'] == foc_store) & (foc_store_time['To_PLACEKEY'] == neib_store)]['Time_mins'].values[0]
                
        visits = group_df[group_df['PLACEKEY'] == neib_store]['visits_by_day']
        if (len(visits) != 0):
            inv_visits += (1/neib_store_time) * visits.values[0]
            inv_visits_exp += (1/np.exp(neib_store_time)) * visits.values[0]
    
    return pd.Series([foc_store, inv_visits, num_reviews_fb_neibmean, num_reviews_ig_neibmean, num_reviews_tw_neibmean,
                      inv_visits_exp, num_reviews_fb_neibmean_exp, num_reviews_ig_neibmean_exp, num_reviews_tw_neibmean_exp])

In [37]:
if unique_neib in brand_visit_local_reviews_list:
    first_neib_metrics = brand_visit_local_reviews[brand_visit_local_reviews['PLACEKEY'].isin(foc_store_first_degree_neibs)][['PLACEKEY', 'visits_by_day','localized_fb_reviews_60_days','localized_ig_reviews_60_days', 
                                                                                                                                'localized_tw_reviews_60_days']].groupby('date').apply(calculate_first_neib_mean_reviews_visits)
else:
    first_neib_metrics = brands_visits[brands_visits['PLACEKEY'].isin(foc_store_first_degree_neibs)][['PLACEKEY', 'visits_by_day']].groupby('date').apply(calculate_first_neib_visits)

In [ ]:
first_neib_metrics = first_neib_metrics.rename(columns={0:'focal_store', 1:'inv_visits', 2: 'num_reviews_fb_neibmean', 3:'num_reviews_ig_neibmean', 4:'num_reviews_tw_neibmean',
                                                        5:'inv_visits_exp', 6:'num_reviews_fb_neibmean_exp', 7: 'num_reviews_ig_neibmean_exp', 8: 'num_reviews_tw_neibmean_exp'})
first_neib_metrics.head()

### Calculating metrics for second degree brands

In [ ]:
foc_store

In [ ]:
foc_store_first_degree_neibs

In [ ]:
# Excluding first degree neighboring brand, all the first degree neighbors brand stores, and calculating the second degree neighbors (less than 1 mile)
second_neib_df = neib_distance_km[neib_distance_km['SRC_PLACEKEY'].isin(foc_store_first_degree_neibs) &
                                (~neib_distance_km['DST_BRAND'].isin(foc_store_all_first_degree_neibs_brands)) &
                                (neib_distance_km['DST_BRAND'] != unique_neib)]
second_neib_df

In [ ]:
second_neib_df_local_reviews = second_neib_df[second_neib_df['DST_BRAND'].isin(brand_visit_local_reviews_list)]
second_neib_df_local_reviews

In [49]:
first_deg_stores = second_neib_df['SRC_PLACEKEY'].unique().tolist()
second_deg_stores = second_neib_df['DST_PLACEKEY'].unique().tolist()

In [50]:
first_deg_stores_local_reviews = second_neib_df_local_reviews['SRC_PLACEKEY'].unique().tolist()
second_deg_stores_local_reviews = second_neib_df_local_reviews['DST_PLACEKEY'].unique().tolist()

In [ ]:
first_deg_stores

In [52]:
def calculate_second_neib_visits(group_df):
    
    inv_visits_secondneibmean = 0
    inv_visits_secondneibmean_exp = 0
    
    for first_deg_store in first_deg_stores:
        inv_visits = 0
        inv_visits_exp = 0
        first_deg_store_dist = foc_store_distance[(foc_store_distance['From_PLACEKEY'] == foc_store) & (foc_store_distance['To_PLACEKEY'] == first_deg_store)]['Distance_km'].values[0]
        
        second_deg_store_placekey = second_neib_df[second_neib_df['SRC_PLACEKEY'] == first_deg_store]['DST_PLACEKEY'].to_list()
        second_deg_store_distance = second_neib_df[second_neib_df['SRC_PLACEKEY'] == first_deg_store]['Distance_Km'].to_list()        
        
        # for i in range(len(second_deg_store_placekey)):
        for i in range(10):
            visits = group_df[group_df['PLACEKEY'] == second_deg_store_placekey[i]]['visits_by_day']
            if (len(visits) != 0):
                inv_visits += (1/second_deg_store_distance[i]) * visits.values[0]
                inv_visits_exp += (1/np.exp(second_deg_store_distance[i])) * visits.values[0]
        
        inv_visits_secondneibmean += (1/first_deg_store_dist) * inv_visits
        inv_visits_secondneibmean_exp += (1/np.exp(first_deg_store_dist)) * inv_visits_exp
    
    return pd.Series([foc_store, inv_visits_secondneibmean, inv_visits_secondneibmean_exp])

In [ ]:
second_neib_brand_stores = brands_visits[brands_visits['PLACEKEY'].isin(second_deg_stores)][['PLACEKEY', 'visits_by_day']]
second_neib_metrics_visits = second_neib_brand_stores.groupby('date').apply(calculate_second_neib_visits)
second_neib_metrics_visits = second_neib_metrics_visits.rename(columns={0: 'focal_store', 1:'inv_visits_secondneibmean', 2:'inv_visits_secondneibmean_exp'}).reset_index()
second_neib_metrics_visits.head()

In [54]:
def calculate_second_neib_mean_reviews(group_df):
    
    num_reviews_fb_secondneibmean = 0
    num_reviews_ig_secondneibmean = 0
    num_reviews_tw_secondneibmean = 0
    num_reviews_fb_secondneibmean_exp = 0
    num_reviews_ig_secondneibmean_exp = 0
    num_reviews_tw_secondneibmean_exp = 0
    
    for first_deg_store in first_deg_stores_local_reviews:
        
        num_reviews_fb_neibmean = 0
        num_reviews_ig_neibmean = 0
        num_reviews_tw_neibmean = 0
        num_reviews_fb_neibmean_exp = 0
        num_reviews_ig_neibmean_exp = 0
        num_reviews_tw_neibmean_exp = 0
        
        first_deg_store_dist = foc_store_distance[(foc_store_distance['From_PLACEKEY'] == foc_store) & (foc_store_distance['To_PLACEKEY'] == first_deg_store)]['Distance_km'].values[0]
        
        second_deg_store_placekey = second_neib_df_local_reviews[second_neib_df_local_reviews['SRC_PLACEKEY'] == first_deg_store]['DST_PLACEKEY'].to_list()
        second_deg_store_distance = second_neib_df_local_reviews[second_neib_df_local_reviews['SRC_PLACEKEY'] == first_deg_store]['Distance_Km'].to_list()
        
        # for i in range(len(second_deg_store_placekey)):
        for i in range(10):
            second_neib_brand_stores_local_reviews = group_df[group_df['PLACEKEY'] == second_deg_store_placekey[i]][['localized_fb_reviews_60_days',
                                                                                                                 'localized_ig_reviews_60_days',
                                                                                                                 'localized_tw_reviews_60_days']]
            fb_reviews = second_neib_brand_stores_local_reviews['localized_fb_reviews_60_days']
            if (len(fb_reviews) != 0):
                num_reviews_fb_neibmean += (1/second_deg_store_distance[i]) * fb_reviews.values[0]
                num_reviews_fb_neibmean_exp += (1/np.exp(second_deg_store_distance[i])) * fb_reviews.values[0]
            
            ig_reviews = second_neib_brand_stores_local_reviews['localized_ig_reviews_60_days']
            if (len(ig_reviews) != 0):
                num_reviews_ig_neibmean += (1/second_deg_store_distance[i]) * ig_reviews.values[0]
                num_reviews_ig_neibmean_exp += (1/np.exp(second_deg_store_distance[i])) * ig_reviews.values[0]
            
            tw_reviews = second_neib_brand_stores_local_reviews['localized_tw_reviews_60_days']
            if (len(tw_reviews) != 0):
                num_reviews_tw_neibmean += (1/second_deg_store_distance[i]) * tw_reviews.values[0]
                num_reviews_tw_neibmean_exp += (1/np.exp(second_deg_store_distance[i])) * tw_reviews.values[0]

        num_reviews_fb_secondneibmean += (1/first_deg_store_dist) * num_reviews_fb_neibmean
        num_reviews_ig_secondneibmean += (1/first_deg_store_dist) * num_reviews_ig_neibmean
        num_reviews_tw_secondneibmean += (1/first_deg_store_dist) * num_reviews_tw_neibmean
        
        num_reviews_fb_secondneibmean_exp += (1/np.exp(first_deg_store_dist)) * num_reviews_fb_neibmean_exp
        num_reviews_ig_secondneibmean_exp += (1/np.exp(first_deg_store_dist)) * num_reviews_ig_neibmean_exp
        num_reviews_tw_secondneibmean_exp += (1/np.exp(first_deg_store_dist)) * num_reviews_tw_neibmean_exp
    
    return pd.Series([foc_store, num_reviews_fb_secondneibmean, num_reviews_ig_secondneibmean, num_reviews_tw_secondneibmean, 
                      num_reviews_fb_secondneibmean_exp, num_reviews_ig_secondneibmean_exp, num_reviews_tw_secondneibmean_exp])

In [ ]:
second_neib_brand_stores_local_reviews = brand_visit_local_reviews[brand_visit_local_reviews['PLACEKEY'].isin(second_deg_stores_local_reviews)][['PLACEKEY',
                                                                                                                                                 'localized_fb_reviews_60_days',
                                                                                                                                                 'localized_ig_reviews_60_days', 
                                                                                                                                                 'localized_tw_reviews_60_days']]
second_neib_metrics_local_reviews = second_neib_brand_stores_local_reviews.groupby('date').apply(calculate_second_neib_mean_reviews)

second_neib_metrics_local_reviews = second_neib_metrics_local_reviews.rename(columns={0: 'focal_store', 1:'num_reviews_fb_secondneibmean', 2:'num_reviews_ig_secondneibmean',
                                                                                      3: 'num_reviews_tw_secondneibmean', 4: 'num_reviews_fb_secondneibmean_exp',
                                                                                      5: 'num_reviews_ig_secondneibmean_exp', 6: 'num_reviews_tw_secondneibmean_exp'}).reset_index()
second_neib_metrics_local_reviews.head()

In [ ]:
second_neib_metrics = pd.merge(left=second_neib_metrics_visits, right=second_neib_metrics_local_reviews, how='inner', on=['date', 'focal_store'])
second_neib_metrics.head()

### Extracting metrics for the specific focal store in terms of local reviews

In [ ]:
brand_visit_local_reviews

In [72]:
# Extracting Local Review Information for the focal store
focal_store_information = brand_visit_local_reviews[brand_visit_local_reviews['PLACEKEY'] == foc_store][['PLACEKEY', 'brand_visitation', 'visits_by_day', 'visits_past_60_days',
                                                                                                         'localized_fb_reviews_60_days', 'localized_ig_reviews_60_days', 'localized_tw_reviews_60_days']]

In [73]:
# Combining distance metrics with the focal store info
focal_store_information['spatial_distance_km'] = spatial_distance_avg
focal_store_information['travel_distance_min'] = travel_distance_avg

In [ ]:
focal_store_information

In [ ]:
pd.merge(left=focal_store_information, right=first_neib_metrics, how='inner', on=['date']).drop('focal_store', axis=1)

In [76]:
# Combining first and second degree neighbor metrics with the focal store info
focal_store_information = pd.merge(left=focal_store_information, right=first_neib_metrics, how='inner', on=['date']).drop('focal_store', axis=1)
focal_store_information = pd.merge(left=focal_store_information, right=second_neib_metrics, how='inner', on=['date']).drop('focal_store', axis=1)
focal_store_information = focal_store_information.sort_values('date').fillna(0)

In [ ]:
focal_store_information.head()